## Queries for Project 

In [1]:
import duckdb
import pandas as pd

# Define paths to CSV files
movies_path = r'C:\Users\marti\OneDrive\Documentos\IMS_EDSA\B.D.F\ml-latest-small\movies.csv'
ratings_path = r'C:\Users\marti\OneDrive\Documentos\IMS_EDSA\B.D.F\ml-latest-small\ratings.csv'
links_path = r'C:\Users\marti\OneDrive\Documentos\IMS_EDSA\B.D.F\ml-latest-small\links.csv'
tags_path = r'C:\Users\marti\OneDrive\Documentos\IMS_EDSA\B.D.F\ml-latest-small\tags.csv'

# Create tables in DuckDB
duckdb.sql(f"CREATE OR REPLACE TABLE movies AS SELECT * FROM read_csv_auto('{movies_path}')")
duckdb.sql(f"CREATE OR REPLACE TABLE ratings AS SELECT * FROM read_csv_auto('{ratings_path}')")
duckdb.sql(f"CREATE OR REPLACE TABLE links AS SELECT * FROM read_csv_auto('{links_path}')")
duckdb.sql(f"CREATE OR REPLACE TABLE tags AS SELECT * FROM read_csv_auto('{tags_path}')")

print("✓ All tables loaded successfully into DuckDB")
print(f"  - movies: {duckdb.sql('SELECT COUNT(*) as count FROM movies').df()['count'][0]} records")
print(f"  - ratings: {duckdb.sql('SELECT COUNT(*) as count FROM ratings').df()['count'][0]} records")
print(f"  - links: {duckdb.sql('SELECT COUNT(*) as count FROM links').df()['count'][0]} records")
print(f"  - tags: {duckdb.sql('SELECT COUNT(*) as count FROM tags').df()['count'][0]} records")

✓ All tables loaded successfully into DuckDB
  - movies: 9742 records
  - ratings: 100836 records
  - links: 9742 records
  - tags: 3683 records


## 1. Movies Analysis - Basic Statistics

In [2]:
# Query 1: Movie genres breakdown - explode and count
result1 = duckdb.sql("""
    SELECT genre, COUNT(*) AS movie_count
    FROM movies,
         UNNEST(STRING_SPLIT(genres, '|')) AS t(genre)
    GROUP BY genre
    ORDER BY movie_count DESC
""").df()

print("QUERY 1: Movies per Genre")
print("=" * 60)
print(result1.to_string(index=False))
print()

# The UNNEST turns a list  like ['Drama', 'Comedy', 'Action'], UNNEST will turn it into three separate rows


QUERY 1: Movies per Genre
             genre  movie_count
             Drama         4361
            Comedy         3756
          Thriller         1894
            Action         1828
           Romance         1596
         Adventure         1263
             Crime         1199
            Sci-Fi          980
            Horror          978
           Fantasy          779
          Children          664
         Animation          611
           Mystery          573
       Documentary          440
               War          382
           Musical          334
           Western          167
              IMAX          158
         Film-Noir           87
(no genres listed)           34



In [3]:
# Query 2: Movies with release year
result2 = duckdb.sql("""
    SELECT 
        CAST(SUBSTR(title, LENGTH(title) - 4, 4) AS INTEGER) AS release_year,
        COUNT(*) AS count_movies,
        COUNT(DISTINCT movieId) AS unique_movies
    FROM movies
    WHERE REGEXP_MATCHES(SUBSTR(title, LENGTH(title) - 4, 4), '^[0-9]{4}$')
    GROUP BY release_year
    ORDER BY release_year DESC
    LIMIT 30
""").df()

print("QUERY 2: Movies by Release Year (Last 30 Years)")
print("=" * 60)
print(result2.to_string(index=False))
print()

QUERY 2: Movies by Release Year (Last 30 Years)
 release_year  count_movies  unique_movies
         2018            41             41
         2017           147            147
         2016           218            218
         2015           274            274
         2014           277            277
         2013           239            239
         2012           232            232
         2011           252            252
         2010           247            247
         2009           282            282
         2008           268            268
         2007           283            283
         2006           295            295
         2005           273            273
         2004           279            279
         2003           279            279
         2002           311            311
         2001           294            294
         2000           283            283
         1999           262            262
         1998           257            257
      

- This query gives you a time‑based lens on your dataset: how many movies per release year are included, which years dominate, and whether the dataset skews toward certain eras.

## 2. Ratings Analysis - Aggregated Insights

In [4]:
# Query 3: Rating distribution
result3 = duckdb.sql("""
    SELECT rating,COUNT(*) as count_ratings, ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM ratings), 2) as percentage
    FROM ratings
    GROUP BY rating
    ORDER BY rating
""").df()

print("QUERY 3: Rating Distribution")
print("=" * 60)
print(result3.to_string(index=False))
print()

QUERY 3: Rating Distribution
 rating  count_ratings  percentage
    0.5           1370        1.36
    1.0           2811        2.79
    1.5           1791        1.78
    2.0           7551        7.49
    2.5           5550        5.50
    3.0          20047       19.88
    3.5          13136       13.03
    4.0          26818       26.60
    4.5           8551        8.48
    5.0          13211       13.10



In [5]:
# Query 4: User engagement - ratings per user
result4 = duckdb.sql("""
    SELECT 
        'Total Users' as metric,
        COUNT(DISTINCT userId) as value
    FROM ratings
    UNION ALL
    SELECT 
        'Average Ratings per User',
        ROUND(AVG(ratings_count))
    FROM (
        SELECT userId, COUNT(*) as ratings_count
        FROM ratings
        GROUP BY userId
    )
    UNION ALL
    SELECT 
        'Max Ratings by One User',
        MAX(ratings_count)
    FROM (
        SELECT userId, COUNT(*) as ratings_count
        FROM ratings
        GROUP BY userId
    )
    UNION ALL
    SELECT 
        'Min Ratings by One User',
        MIN(ratings_count)
    FROM (
        SELECT userId, COUNT(*) as ratings_count
        FROM ratings
        GROUP BY userId
    )
""").df()

print("QUERY 4: User Engagement Metrics")
print("=" * 60)
print(result4.to_string(index=False))
print()

QUERY 4: User Engagement Metrics
                  metric  value
             Total Users  610.0
Average Ratings per User  165.0
 Max Ratings by One User 2698.0
 Min Ratings by One User   20.0



It shows that there are 610 distinct users who contributed ratings, which defines the size of the active user base. On average, each user rated about 165 movies, indicating a reasonably engaged community rather than casual one‑off interactions. However, the spread of activity is very uneven: the most active user rated 2,698 movies, while the least active user rated only 20 movies. This imbalance highlights a common pattern in recommendation datasets, where a small group of “super‑users” generate a large proportion of the data, while many others contribute only minimally. The key takeaway is that although the dataset is rich enough to support recommender system experiments, the variability in user engagement must be considered carefully, since highly active users can disproportionately influence results while low‑activity users may lack sufficient data for personalised recommendations.


## 3. Movie-Rating Relationships

In [6]:
# Query 5: Top-rated movies (with minimum ratings threshold)
result5 = duckdb.sql("""
    SELECT 
        m.movieId,
        m.title,
        ROUND(AVG(r.rating), 2) as avg_rating,
        COUNT(r.rating) as total_ratings,
        MIN(r.rating) as min_rating,
        MAX(r.rating) as max_rating,
        ROUND(STDDEV(r.rating), 2) as rating_stddev
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.movieId, m.title
    HAVING COUNT(r.rating) >= 50
    ORDER BY avg_rating DESC
    LIMIT 20
""").df()

print("QUERY 5: Top 20 Highest-Rated Movies (50+ ratings)")
print("=" * 60)
print(result5.to_string(index=False))
print()

QUERY 5: Top 20 Highest-Rated Movies (50+ ratings)
 movieId                                                                          title  avg_rating  total_ratings  min_rating  max_rating  rating_stddev
     318                                               Shawshank Redemption, The (1994)        4.43            317         1.0         5.0           0.71
     858                                                          Godfather, The (1972)        4.29            192         1.0         5.0           0.90
     750    Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)        4.27             97         1.0         5.0           0.81
    1276                                                          Cool Hand Luke (1967)        4.27             57         2.5         5.0           0.62
    2959                                                              Fight Club (1999)        4.27            218         0.5         5.0           0.86
     904                 

In [7]:
# Query 6: Most popular movies (by number of ratings)
result6 = duckdb.sql("""
    SELECT 
        m.movieId,
        m.title,
        COUNT(r.rating) as total_ratings,
        ROUND(AVG(r.rating), 2) as avg_rating
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.movieId, m.title
    ORDER BY total_ratings DESC
    LIMIT 20
""").df()

print("QUERY 6: Top 20 Most Rated Movies (by popularity)")
print("=" * 60)
print(result6.to_string(index=False))
print()

QUERY 6: Top 20 Most Rated Movies (by popularity)
 movieId                                                                          title  total_ratings  avg_rating
     356                                                            Forrest Gump (1994)            329        4.16
     318                                               Shawshank Redemption, The (1994)            317        4.43
     296                                                            Pulp Fiction (1994)            307        4.20
     593                                               Silence of the Lambs, The (1991)            279        4.16
    2571                                                             Matrix, The (1999)            278        4.19
     260                                      Star Wars: Episode IV - A New Hope (1977)            251        4.23
     480                                                           Jurassic Park (1993)            238        3.75
     110                      

In [8]:
# Query 7: Genre ratings comparison (fixed)
result7 = duckdb.sql("""
    SELECT
        t.genre AS genre,
        COUNT(r.rating) AS total_ratings,
        ROUND(AVG(r.rating), 2) AS avg_rating,
        ROUND(MAX(r.rating), 2) AS max_rating,
        ROUND(MIN(r.rating), 2) AS min_rating
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    , UNNEST(STRING_SPLIT(m.genres, '|')) AS t(genre)
    GROUP BY t.genre
    ORDER BY avg_rating DESC
""").df()

print("QUERY 7: Average Rating by Genre")
print("=" * 60)
print(result7.to_string(index=False))
print()

QUERY 7: Average Rating by Genre
             genre  total_ratings  avg_rating  max_rating  min_rating
         Film-Noir            870        3.92         5.0         0.5
               War           4859        3.81         5.0         0.5
       Documentary           1219        3.80         5.0         0.5
             Crime          16681        3.66         5.0         0.5
             Drama          41928        3.66         5.0         0.5
         Animation           6988        3.63         5.0         0.5
           Mystery           7674        3.63         5.0         0.5
              IMAX           4145        3.62         5.0         0.5
           Western           1930        3.58         5.0         0.5
           Musical           4138        3.56         5.0         0.5
           Romance          18124        3.51         5.0         0.5
         Adventure          24161        3.51         5.0         0.5
          Thriller          26452        3.49         5.0

- Film-Noir has the highest average rating (3.92), though with relatively few ratings (870).
- War and Documentary also score highly (3.81 and 3.80), suggesting users tend to rate these genres more favourably.
- Drama dominates in volume (41,928 ratings) with a solid average of 3.66.
- Comedy and Horror have very large rating counts but lower averages (3.38 and 3.26), showing they’re popular but not always rated as highly.
- Every genre shows a rating range from 0.5 (minimum) to 5.0 (maximum), which reflects the full rating scale in MovieLens

## 4. Tags Analysis

In [9]:
# Query 8: Most common tags
result8 = duckdb.sql("""
    SELECT 
        tag,
        COUNT(*) as tag_count,
        COUNT(DISTINCT movieId) as movies_with_tag,
        COUNT(DISTINCT userId) as users_who_tagged
    FROM tags
    GROUP BY tag
    ORDER BY tag_count DESC
    LIMIT 25
""").df()

print("QUERY 8: Top 25 Most Common Tags")
print("=" * 60)
print(result8.to_string(index=False))
print()

QUERY 8: Top 25 Most Common Tags
               tag  tag_count  movies_with_tag  users_who_tagged
  In Netflix queue        131              131                 1
       atmospheric         36               32                 9
         superhero         24               22                 7
 thought-provoking         24               19                 7
             funny         23               21                 9
           surreal         23               21                 6
            Disney         23               22                 3
          religion         22               22                 2
       dark comedy         21               16                 9
        psychology         21               19                 7
            quirky         21               21                 5
            sci-fi         21               17                10
          suspense         20               18                 8
      twist ending         19               15           

In [10]:
# Query 9: Movies with the most tags
result9 = duckdb.sql("""
    SELECT 
        m.movieId,
        m.title,
        COUNT(t.tag) as tag_count,
        COUNT(DISTINCT t.userId) as distinct_users_tagged
    FROM movies m
    LEFT JOIN tags t ON m.movieId = t.movieId
    GROUP BY m.movieId, m.title
    HAVING COUNT(t.tag) > 0
    ORDER BY tag_count DESC
    LIMIT 20
""").df()

print("QUERY 9: Top 20 Most Tagged Movies")
print("=" * 60)
print(result9.to_string(index=False))
print()

QUERY 9: Top 20 Most Tagged Movies
 movieId                                                          title  tag_count  distinct_users_tagged
     296                                            Pulp Fiction (1994)        181                      4
    2959                                              Fight Club (1999)         54                      4
     924                                   2001: A Space Odyssey (1968)         41                      2
     293 Léon: The Professional (a.k.a. The Professional) (Léon) (1994)         35                      3
    7361                   Eternal Sunshine of the Spotless Mind (2004)         34                      5
    1732                                       Big Lebowski, The (1998)         32                      2
    4878                                            Donnie Darko (2001)         29                      5
     260                      Star Wars: Episode IV - A New Hope (1977)         26                     10
   79132   

## 5. Complex Multi-Table Joins

In [11]:
# Query 10: Complete movie profile (movies + ratings + tags)
result10 = duckdb.sql("""
    SELECT 
        m.movieId,
        m.title,
        COUNT(DISTINCT r.userId) as num_raters,
        ROUND(AVG(r.rating), 2) as avg_rating,
        COUNT(DISTINCT t.tag) as unique_tags,
        COUNT(DISTINCT t.userId) as num_taggers,
        STRING_AGG(DISTINCT SUBSTR(m.genres, 1, 50), ', ') as genres
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    LEFT JOIN tags t ON m.movieId = t.movieId
    GROUP BY m.movieId, m.title
    ORDER BY num_raters DESC
    LIMIT 25
""").df()

print("QUERY 10: Complete Movie Profiles (Top 25 by Raters)")
print("=" * 60)
print(result10.to_string(index=False))
print()

QUERY 10: Complete Movie Profiles (Top 25 by Raters)
 movieId                                                                          title  num_raters  avg_rating  unique_tags  num_taggers                                      genres
     356                                                            Forrest Gump (1994)         329        4.16            9            3                    Comedy|Drama|Romance|War
     318                                               Shawshank Redemption, The (1994)         317        4.43            4            2                                 Crime|Drama
     296                                                            Pulp Fiction (1994)         307        4.20          173            4                 Comedy|Crime|Drama|Thriller
     593                                               Silence of the Lambs, The (1991)         279        4.16            6            2                       Crime|Horror|Thriller
    2571                             

In [12]:
# Query 11: IMDb Link analysis - movies with external links
result11 = duckdb.sql("""
    SELECT 
        m.movieId,
        m.title,
        l.imdbId,
        l.tmdbId,
        COUNT(DISTINCT r.userId) as num_ratings,
        ROUND(AVG(r.rating), 2) as avg_rating
    FROM movies m
    LEFT JOIN links l ON m.movieId = l.movieId
    LEFT JOIN ratings r ON m.movieId = r.movieId
    WHERE l.imdbId IS NOT NULL
    GROUP BY m.movieId, m.title, l.imdbId, l.tmdbId
    ORDER BY num_ratings DESC
    LIMIT 20
""").df()

print("QUERY 11: Movies with IMDb/TMDB Links (Top 20 by Ratings)")
print("=" * 60)
print(result11.to_string(index=False))
print()

QUERY 11: Movies with IMDb/TMDB Links (Top 20 by Ratings)
 movieId                                                                          title  imdbId  tmdbId  num_ratings  avg_rating
     356                                                            Forrest Gump (1994) 0109830      13          329        4.16
     318                                               Shawshank Redemption, The (1994) 0111161     278          317        4.43
     296                                                            Pulp Fiction (1994) 0110912     680          307        4.20
     593                                               Silence of the Lambs, The (1991) 0102926     274          279        4.16
    2571                                                             Matrix, The (1999) 0133093     603          278        4.19
     260                                      Star Wars: Episode IV - A New Hope (1977) 0076759      11          251        4.23
     480                               

## 6. Advanced Analytics

In [13]:
# Query 12: Correlation between popularity and rating quality
result12 = duckdb.sql("""
    SELECT 
        CASE 
            WHEN rating_count < 10 THEN '1-10 ratings'
            WHEN rating_count < 50 THEN '10-50 ratings'
            WHEN rating_count < 100 THEN '50-100 ratings'
            WHEN rating_count < 500 THEN '100-500 ratings'
            ELSE '500+ ratings'
        END as popularity_bucket,
        COUNT(*) as movie_count,
        ROUND(AVG(avg_rating), 2) as avg_rating,
        ROUND(MIN(avg_rating), 2) as min_rating,
        ROUND(MAX(avg_rating), 2) as max_rating
    FROM (
        SELECT 
            m.movieId,
            COUNT(r.rating) as rating_count,
            AVG(r.rating) as avg_rating
        FROM movies m
        JOIN ratings r ON m.movieId = r.movieId
        GROUP BY m.movieId
    )
    GROUP BY popularity_bucket
    ORDER BY 
        CASE popularity_bucket
            WHEN '1-10 ratings' THEN 1
            WHEN '10-50 ratings' THEN 2
            WHEN '50-100 ratings' THEN 3
            WHEN '100-500 ratings' THEN 4
            WHEN '500+ ratings' THEN 5
        END
""").df()

print("QUERY 12: Popularity vs Rating Quality Analysis")
print("=" * 60)
print(result12.to_string(index=False))
print()

QUERY 12: Popularity vs Rating Quality Analysis
popularity_bucket  movie_count  avg_rating  min_rating  max_rating
     1-10 ratings         7455        3.21        0.50        5.00
    10-50 ratings         1819        3.37        1.35        4.59
   50-100 ratings          312        3.59        2.21        4.27
  100-500 ratings          138        3.82        2.91        4.43



In [17]:
# Query 13: Genre diversity - which genres pair most often
result13 = duckdb.sql("""
    WITH genre_pairs AS (
        SELECT 
            m.movieId,
            UNNEST(STRING_SPLIT(m.genres, '|')) as genre
        FROM movies m
    )
    SELECT 
        g1.genre as genre_1,
        g2.genre as genre_2,
        COUNT(*) as movies_with_both_genres
    FROM genre_pairs g1
    JOIN genre_pairs g2 ON g1.movieId = g2.movieId AND g1.genre < g2.genre
    GROUP BY g1.genre, g2.genre
    ORDER BY movies_with_both_genres DESC
    LIMIT 15
""").df()

print("""duckdb.sql(
    WITH genre_pairs AS (
        SELECT 
            m.movieId,
            UNNEST(STRING_SPLIT(m.genres, '|')) as genre
        FROM movies m
    )
    SELECT 
        g1.genre as genre_1,
        g2.genre as genre_2,
        COUNT(*) as movies_with_both_genres
    FROM genre_pairs g1
    JOIN genre_pairs g2 ON g1.movieId = g2.movieId AND g1.genre < g2.genre
    GROUP BY g1.genre, g2.genre
    ORDER BY movies_with_both_genres DESC
    LIMIT 15
""")
print("QUERY: Most Common Genre Combinations")
print("=" * 60)
print(result13.to_string(index=False))
print()

duckdb.sql(
    WITH genre_pairs AS (
        SELECT 
            m.movieId,
            UNNEST(STRING_SPLIT(m.genres, '|')) as genre
        FROM movies m
    )
    SELECT 
        g1.genre as genre_1,
        g2.genre as genre_2,
        COUNT(*) as movies_with_both_genres
    FROM genre_pairs g1
    JOIN genre_pairs g2 ON g1.movieId = g2.movieId AND g1.genre < g2.genre
    GROUP BY g1.genre, g2.genre
    ORDER BY movies_with_both_genres DESC
    LIMIT 15

QUERY: Most Common Genre Combinations
  genre_1   genre_2  movies_with_both_genres
   Comedy     Drama                     1013
    Drama   Romance                      934
   Comedy   Romance                      884
    Drama  Thriller                      832
   Action  Thriller                      655
    Crime     Drama                      637
   Action Adventure                      610
    Crime  Thriller                      569
   Action     Drama                      529
   Horror  Thriller                      459
   A

In [26]:
import duckdb

result14 = duckdb.sql("""
    SELECT 
        genre,
        t.tag,
        COUNT(*) AS frequency
    FROM movies m
    JOIN tags t ON m.movieId = t.movieId
    CROSS JOIN UNNEST(STRING_SPLIT(m.genres, '|')) AS genre
    GROUP BY genre, t.tag
    ORDER BY genre, frequency DESC
    LIMIT 10
""").df()

print("QUERY 14: Top Tags per Genre (First 30 Results)")
print("=" * 60)
print(result14.to_string(index=False))

QUERY 14: Top Tags per Genre (First 30 Results)
                           genre         tag  frequency
{'unnest': '(no genres listed)'}       sweet          1
{'unnest': '(no genres listed)'} understated          1
{'unnest': '(no genres listed)'}      quirky          1
            {'unnest': 'Action'}   superhero         23
            {'unnest': 'Action'}      sci-fi         16
            {'unnest': 'Action'}      action         11
            {'unnest': 'Action'}  comic book         11
            {'unnest': 'Action'}       anime          8
            {'unnest': 'Action'}       funny          8
            {'unnest': 'Action'}       space          8


## 7. User-Centric Insights

In [ ]:
# Query 15: Most active users - rating and tagging behavior
result15 = duckdb.sql("""
    SELECT 
        r.userId,
        COUNT(r.rating) as ratings_count,
        ROUND(AVG(r.rating), 2) as avg_rating_given,
        COUNT(DISTINCT r.movieId) as distinct_movies_rated,
        COUNT(DISTINCT t.tag) as tags_given,
        COUNT(DISTINCT t.movieId) as movies_tagged
    FROM ratings r
    LEFT JOIN tags t ON r.userId = t.userId
    GROUP BY r.userId
    ORDER BY ratings_count DESC
    LIMIT 10
""").df()

print("QUERY: Top 10 Most Active Users (Raters & Taggers)")
print("=" * 60)
print(result15.to_string(index=False))
print()

duckdb.sql(
    SELECT 
        r.userId,
        COUNT(r.rating) as ratings_count,
        ROUND(AVG(r.rating), 2) as avg_rating_given,
        COUNT(DISTINCT r.movieId) as distinct_movies_rated,
        COUNT(DISTINCT t.tag) as tags_given,
        COUNT(DISTINCT t.movieId) as movies_tagged
    FROM ratings r
    LEFT JOIN tags t ON r.userId = t.userId
    GROUP BY r.userId
    ORDER BY ratings_count DESC
    LIMIT 10
) 
QUERY: Top 10 Most Active Users (Raters & Taggers)
 userId  ratings_count  avg_rating_given  distinct_movies_rated  tags_given  movies_tagged
    474        3176756              3.40                   2108         595           1235
    599         800394              2.64                   2478         271              5
    477         168000              3.74                    600         215             68
    567         166320              2.25                    385         192            109
     62         135420              4.08                    366     

In [20]:
# Query 16: User rating patterns - strict vs lenient raters
result16 = duckdb.sql("""
    SELECT 
        user_rating_pattern,
        COUNT(*) as num_users,
        ROUND(AVG(avg_rating), 2) as avg_rating_per_user,
        ROUND(AVG(rating_count), 0) as avg_ratings_per_user
    FROM (
        SELECT 
            r.userId,
            COUNT(r.rating) as rating_count,
            AVG(r.rating) as avg_rating,
            CASE 
                WHEN AVG(r.rating) < 2.5 THEN 'Strict Raters'
                WHEN AVG(r.rating) < 3.5 THEN 'Moderate Raters'
                WHEN AVG(r.rating) < 4.0 THEN 'Generous Raters'
                ELSE 'Very Generous Raters'
            END as user_rating_pattern
        FROM ratings r
        GROUP BY r.userId
    )
    GROUP BY user_rating_pattern
    ORDER BY avg_rating_per_user
""").df()

print("QUERY 16: User Rating Patterns (Strict vs Generous Raters)")
print("=" * 60)
print(result16.to_string(index=False))
print()

QUERY 16: User Rating Patterns (Strict vs Generous Raters)
 user_rating_pattern  num_users  avg_rating_per_user  avg_ratings_per_user
       Strict Raters         10                 2.20                 232.0
     Moderate Raters        202                 3.19                 228.0
     Generous Raters        245                 3.75                 156.0
Very Generous Raters        153                 4.22                  93.0

